<a href="https://colab.research.google.com/github/rrfsantos/Desafios-NLP/blob/main/Desafio-NLP--NER/insert_label_iob_convert_spacy_arquivos_separados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install --user spacy==3.0.0

     |████████████████████████████████| 12.7MB 293kB/s 
     |████████████████████████████████| 9.1MB 37.9MB/s 
     |████████████████████████████████| 460kB 47.0MB/s 
     |████████████████████████████████| 1.1MB 47.8MB/s 
     |████████████████████████████████| 122kB 38.3MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-cp37-none-any.whl size=107098 sha256=0ff5f3c4f3cb7215487113fc81e1cb3cadffc0bd7c48b2ccef77aab572cac188
  Stored in directory: /root/.cache/pip/wheels/18/88/7c/f06dabd5e9cabe02d2269167bcacbbf9b47d0c0ff7d6ebcb78
Successfully built smart-open
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
import pandas as pd
import warnings
import glob
import csv

import spacy

warnings.filterwarnings("ignore")
print(spacy.__version__)

3.0.0


In [5]:
import os
from google.colab import drive
drive.mount('/content/drive')
workdir_path = '/content/drive/My Drive/desafio 2' #MODIFICAR!!! #Caminho para o seu workspace
os.chdir(workdir_path)

Mounted at /content/drive


## Ler arquivos IOB e incluir o nome da classe à tag

In [3]:
dirs = ['s800','linnaeus','BC5CDR-disease','NCBI-disease','BC2GM','JNLPBA','BC4CHEMD','BC5CDR-chem']

#### Arquivos de treino

In [8]:
for dir in dirs:
    df = pd.read_csv('NERdata/' + dir + '/train.tsv', sep='\t', header=None, names=['token','tag'],
                         quoting=csv.QUOTE_NONE, error_bad_lines=False, skipinitialspace = True)
    if (dir == 's800' or dir == 'linnaeus'):
      df['tag'] = df['tag'].replace(['B','I'],['B-SPECIES','I-SPECIES'])
    elif (dir == 'BC5CDR-disease' or dir == 'NCBI-disease'):
      df['tag'] = df['tag'].replace(['B','I'],['B-DISEASE','I-DISEASE'])
    elif (dir == 'BC2GM' or dir == 'JNLPBA'):
      df['tag'] = df['tag'].replace(['B','I'],['B-DRUG_PROTEIN','I-DRUG_PROTEIN'])
    else:
      df['tag'] = df['tag'].replace(['B','I'],['B-CHEMICALS','I-CHEMICALS'])

    df.dropna(inplace=True)

    df.to_csv('IOBdata/iob_train_' + dir + '.iob', sep='\t', index=False, header=False)  

## Converter arquivos em formato IOB para o formato binário .spacy

In [9]:
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_train_s800.iob ./SPACYdata

2021-04-10 19:28:28.505040: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10 sentences into a document.
ℹ Segmenting sentences with sentencizer. (Use `-b model` for improved
parser-based sentence segmentation.)
✔ Generated output file (685 documents):
SPACYdata/iob_train_s800.spacy


In [10]:
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_train_linnaeus.iob ./SPACYdata

2021-04-10 19:31:40.287384: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10 sentences into a document.
ℹ Segmenting sentences with sentencizer. (Use `-b model` for improved
parser-based sentence segmentation.)
✔ Generated output file (1335 documents):
SPACYdata/iob_train_linnaeus.spacy


In [18]:
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_train_BC5CDR-disease.iob ./SPACYdata

2021-04-11 01:48:28.255353: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10 sentences into a document.
ℹ Segmenting sentences with sentencizer. (Use `-b model` for improved
parser-based sentence segmentation.)
✔ Generated output file (582 documents):
SPACYdata/iob_train_BC5CDR-disease.spacy


In [19]:
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_train_NCBI-disease.iob ./SPACYdata

2021-04-11 01:50:34.079466: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10 sentences into a document.
ℹ Segmenting sentences with sentencizer. (Use `-b model` for improved
parser-based sentence segmentation.)
✔ Generated output file (590 documents):
SPACYdata/iob_train_NCBI-disease.spacy


In [13]:
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_train_BC2GM.iob ./SPACYdata

2021-04-10 19:45:13.022844: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10 sentences into a document.
ℹ Segmenting sentences with sentencizer. (Use `-b model` for improved
parser-based sentence segmentation.)
✔ Generated output file (1510 documents):
SPACYdata/iob_train_BC2GM.spacy


In [14]:
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_train_JNLPBA.iob ./SPACYdata

2021-04-10 20:08:47.972433: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10 sentences into a document.
ℹ Segmenting sentences with sentencizer. (Use `-b model` for improved
parser-based sentence segmentation.)
✔ Generated output file (1610 documents):
SPACYdata/iob_train_JNLPBA.spacy


In [16]:
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_train_BC5CDR-chem.iob ./SPACYdata

2021-04-10 21:47:08.643617: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10 sentences into a document.
ℹ Segmenting sentences with sentencizer. (Use `-b model` for improved
parser-based sentence segmentation.)
✔ Generated output file (582 documents):
SPACYdata/iob_train_BC5CDR-chem.spacy


In [17]:
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_train_BC4CHEMD.iob ./SPACYdata

2021-04-10 21:49:14.103205: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10 sentences into a document.
ℹ Segmenting sentences with sentencizer. (Use `-b model` for improved
parser-based sentence segmentation.)
✔ Generated output file (3732 documents):
SPACYdata/iob_train_BC4CHEMD.spacy


#### Arquivos de teste

In [4]:
for dir in dirs:
    df = pd.read_csv('NERdata/' + dir + '/test.tsv', sep='\t', header=None, names=['token','tag'],
                         quoting=csv.QUOTE_NONE, error_bad_lines=False, skipinitialspace = True)
    if (dir == 's800' or dir == 'linnaeus'):
      df['tag'] = df['tag'].replace(['B','I'],['B-SPECIES','I-SPECIES'])
    elif (dir == 'BC5CDR-disease' or dir == 'NCBI-disease'):
      df['tag'] = df['tag'].replace(['B','I'],['B-DISEASE','I-DISEASE'])
    elif (dir == 'BC2GM' or dir == 'JNLPBA'):
      df['tag'] = df['tag'].replace(['B','I'],['B-DRUG_PROTEIN','I-DRUG_PROTEIN'])
    else:
      df['tag'] = df['tag'].replace(['B','I'],['B-CHEMICALS','I-CHEMICALS'])

    df.dropna(inplace=True)

    df.to_csv('IOBdata/iob_test_' + dir + '.iob', sep='\t', index=False, header=False)  

In [5]:
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_test_s800.iob ./SPACYdata

2021-04-11 02:36:45.901932: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10 sentences into a document.
ℹ Segmenting sentences with sentencizer. (Use `-b model` for improved
parser-based sentence segmentation.)
✔ Generated output file (202 documents):
SPACYdata/iob_test_s800.spacy


In [6]:
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_test_linnaeus.iob ./SPACYdata

2021-04-11 02:37:07.072339: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10 sentences into a document.
ℹ Segmenting sentences with sentencizer. (Use `-b model` for improved
parser-based sentence segmentation.)
✔ Generated output file (801 documents):
SPACYdata/iob_test_linnaeus.spacy


In [7]:
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_test_BC5CDR-disease.iob ./SPACYdata

2021-04-11 02:46:23.811542: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10 sentences into a document.
ℹ Segmenting sentences with sentencizer. (Use `-b model` for improved
parser-based sentence segmentation.)
✔ Generated output file (650 documents):
SPACYdata/iob_test_BC5CDR-disease.spacy


In [8]:
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_test_NCBI-disease.iob ./SPACYdata

2021-04-11 02:51:17.162749: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10 sentences into a document.
ℹ Segmenting sentences with sentencizer. (Use `-b model` for improved
parser-based sentence segmentation.)
✔ Generated output file (98 documents):
SPACYdata/iob_test_NCBI-disease.spacy


In [9]:
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_test_BC2GM.iob ./SPACYdata

2021-04-11 02:51:27.137633: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10 sentences into a document.
ℹ Segmenting sentences with sentencizer. (Use `-b model` for improved
parser-based sentence segmentation.)
✔ Generated output file (611 documents):
SPACYdata/iob_test_BC2GM.spacy


In [10]:
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_test_JNLPBA.iob ./SPACYdata

2021-04-11 02:58:02.086551: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10 sentences into a document.
ℹ Segmenting sentences with sentencizer. (Use `-b model` for improved
parser-based sentence segmentation.)
✔ Generated output file (429 documents):
SPACYdata/iob_test_JNLPBA.spacy


In [11]:
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_test_BC5CDR-chem.iob ./SPACYdata

2021-04-11 03:01:19.245979: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10 sentences into a document.
ℹ Segmenting sentences with sentencizer. (Use `-b model` for improved
parser-based sentence segmentation.)
✔ Generated output file (650 documents):
SPACYdata/iob_test_BC5CDR-chem.spacy


In [12]:
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_test_BC4CHEMD.iob ./SPACYdata

2021-04-11 03:05:44.651508: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10 sentences into a document.
ℹ Segmenting sentences with sentencizer. (Use `-b model` for improved
parser-based sentence segmentation.)
✔ Generated output file (3238 documents):
SPACYdata/iob_test_BC4CHEMD.spacy


#### Arquivos de Validação

In [6]:
for dir in dirs:
    df = pd.read_csv('NERdata/' + dir + '/devel.tsv', sep='\t', header=None, names=['token','tag'],
                         quoting=csv.QUOTE_NONE, error_bad_lines=False, skipinitialspace = True)
    if (dir == 's800' or dir == 'linnaeus'):
      df['tag'] = df['tag'].replace(['B','I'],['B-SPECIES','I-SPECIES'])
    elif (dir == 'BC5CDR-disease' or dir == 'NCBI-disease'):
      df['tag'] = df['tag'].replace(['B','I'],['B-DISEASE','I-DISEASE'])
    elif (dir == 'BC2GM' or dir == 'JNLPBA'):
      df['tag'] = df['tag'].replace(['B','I'],['B-DRUG_PROTEIN','I-DRUG_PROTEIN'])
    else:
      df['tag'] = df['tag'].replace(['B','I'],['B-CHEMICALS','I-CHEMICALS'])

    df.dropna(inplace=True)

    df.to_csv('IOBdata/iob_devel_' + dir + '.iob', sep='\t', index=False, header=False)  

In [14]:
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_devel_s800.iob ./SPACYdata

2021-04-11 06:33:16.744773: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10 sentences into a document.
ℹ Segmenting sentences with sentencizer. (Use `-b model` for improved
parser-based sentence segmentation.)
✔ Generated output file (101 documents):
SPACYdata/iob_devel_s800.spacy


In [15]:
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_devel_linnaeus.iob ./SPACYdata

2021-04-11 06:33:25.828508: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10 sentences into a document.
ℹ Segmenting sentences with sentencizer. (Use `-b model` for improved
parser-based sentence segmentation.)
✔ Generated output file (485 documents):
SPACYdata/iob_devel_linnaeus.spacy


In [16]:
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_devel_BC5CDR-disease.iob ./SPACYdata

2021-04-11 06:35:25.229959: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10 sentences into a document.
ℹ Segmenting sentences with sentencizer. (Use `-b model` for improved
parser-based sentence segmentation.)
✔ Generated output file (592 documents):
SPACYdata/iob_devel_BC5CDR-disease.spacy


In [17]:
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_devel_NCBI-disease.iob ./SPACYdata

2021-04-11 06:39:19.132970: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10 sentences into a document.
ℹ Segmenting sentences with sentencizer. (Use `-b model` for improved
parser-based sentence segmentation.)
✔ Generated output file (96 documents):
SPACYdata/iob_devel_NCBI-disease.spacy


In [18]:
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_devel_BC2GM.iob ./SPACYdata

2021-04-11 06:39:28.778502: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10 sentences into a document.
ℹ Segmenting sentences with sentencizer. (Use `-b model` for improved
parser-based sentence segmentation.)
✔ Generated output file (313 documents):
SPACYdata/iob_devel_BC2GM.spacy


In [19]:
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_devel_JNLPBA.iob ./SPACYdata

2021-04-11 06:40:21.673695: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10 sentences into a document.
ℹ Segmenting sentences with sentencizer. (Use `-b model` for improved
parser-based sentence segmentation.)
✔ Generated output file (426 documents):
SPACYdata/iob_devel_JNLPBA.spacy


In [20]:
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_devel_BC5CDR-chem.iob ./SPACYdata

2021-04-11 06:43:56.395318: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10 sentences into a document.
ℹ Segmenting sentences with sentencizer. (Use `-b model` for improved
parser-based sentence segmentation.)
✔ Generated output file (592 documents):
SPACYdata/iob_devel_BC5CDR-chem.spacy


In [ ]:
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_devel_BC4CHEMD.iob ./SPACYdata

2021-04-11 12:55:37.256668: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10 sentences into a document.
ℹ Segmenting sentences with sentencizer. (Use `-b model` for improved
parser-based sentence segmentation.)
